In [1]:
import pandas as pd
import numpy as np
import ast
from datetime import datetime, timezone, timedelta
import geopy.geocoders
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy import distance

## Traffic Alerts

In [2]:
trafficAlertsDf = pd.read_csv('C:/Users/verav/Documents/Projekt/Python_projekt/udalostiVSE.csv', encoding = 'UTF-8')

In [3]:
trafficAlertsDf.head(5)

,type,properties.uuid,properties.pubMillis,properties.type,properties.subtype,properties.roadType,properties.street,properties.city,properties.country,properties.confidence,...,properties.nThumbsUp,properties.reportRating,properties.reportDescription,properties.longitude,properties.latitude,properties.magvar,properties.objectid,properties.globalid,geometry.type,geometry.coordinates
0,Feature,f5987a0c-c5e0-3064-ad5d-9d58b296b1d9,2020/07/12 11:35:50+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,NaN,U Dráhy,Ostopovice,EZ,0,...,NaN,0,NaN,NaN,NaN,0,1,{BDAFA7D7-FAA9-31A1-ABEA-A33DC1F91F17},Point,"[16.544367, 49.16387]"
1,Feature,da8343e6-c5f7-3478-a969-47fb8460c2bb,2020/07/13 03:20:19+00,WEATHERHAZARD,HAZARD_ON_ROAD_CONSTRUCTION,3.0,D1,NaN,EZ,5,...,NaN,3,NaN,NaN,NaN,338,2,{915B7FDD-4E94-2B9D-813F-BB102547C5F6},Point,"[16.417239, 49.209704]"
2,Feature,f5957426-0522-3b9d-b6ba-81c44c8aa182,2020/07/12 10:33:18+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,NaN,Průhon,Brno,EZ,1,...,NaN,0,NaN,NaN,NaN,0,3,{78AE6F8D-46B6-3C7D-65AD-606B16C881E6},Point,"[16.54785, 49.20859]"
3,Feature,84b2780e-43bc-3883-81f9-a54a451efcbb,2020/07/12 15:09:24+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,NaN,NaN,Brno,EZ,0,...,NaN,0,NaN,NaN,NaN,0,4,{39CFFD83-039F-C700-646A-85DF84936E25},Point,"[16.608269, 49.18394]"
4,Feature,c2b5e9f5-8eef-3e08-9092-ba52f4726a62,2020/07/12 10:51:41+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,NaN,NaN,Brno,EZ,0,...,NaN,0,NaN,NaN,NaN,0,5,{DD1FB7D7-1967-F229-3D7A-E22413453428},Point,"[16.565195, 49.168989]"


In [4]:
trafficAlertsDf = trafficAlertsDf.drop(columns = ['type'])
trafficAlertsDf.columns = trafficAlertsDf.columns.str.replace('properties.','')
trafficAlertsDf= trafficAlertsDf.rename(columns={'geometry.type':'coordType','geometry.coordinates':'coordinates','uuid':'alertID'})

In [5]:
trafficAlertsDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86605 entries, 0 to 86604
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   alertID            86605 non-null  object 
 1   pubMillis          86605 non-null  object 
 2   type               86605 non-null  object 
 3   subtype            81156 non-null  object 
 4   roadType           83023 non-null  float64
 5   street             82801 non-null  object 
 6   city               56269 non-null  object 
 7   country            86431 non-null  object 
 8   confidence         86605 non-null  int64  
 9   reliability        86605 non-null  int64  
 10  nThumbsUp          0 non-null      float64
 11  reportRating       86605 non-null  int64  
 12  reportDescription  0 non-null      float64
 13  longitude          0 non-null      float64
 14  latitude           0 non-null      float64
 15  magvar             86605 non-null  int64  
 16  objectid           866

In [6]:
trafficAlertsDf = trafficAlertsDf.drop(['roadType','country','nThumbsUp','reportRating','reportDescription', 'longitude', 'latitude','magvar','objectid','globalid'],axis = 1)

zpracování formátu data a času

In [7]:
def dateTime(vstup):
    return datetime.strptime(vstup,'%Y/%m/%d %H:%M:%S+%z')

In [8]:
trafficAlertsDf['datumCas'] = pd.to_datetime(trafficAlertsDf['pubMillis']).dt.tz_convert(tz = 'Europe/Berlin')

In [9]:
trafficAlertsDf['datum'] = trafficAlertsDf['datumCas'].dt.date

In [10]:
trafficAlertsDf['weekdays'] = trafficAlertsDf['datumCas'].dt.day_name()

In [11]:
trafficAlertsDf['hours'] = trafficAlertsDf['datumCas'].dt.hour

In [12]:
trafficAlertsDf['minutes'] = trafficAlertsDf['datumCas'].dt.minute

In [13]:
def quarter(vstup):
    if vstup <= 15:
        return '00'
    elif vstup <= 30:
        return '15'
    elif vstup <= 45:
        return '30'
    else:
        return '45'

In [14]:
trafficAlertsDf['quarters']=trafficAlertsDf['minutes'].apply(quarter)

přehození sloupců, aby byly souřadnice na konci

In [15]:
trafficAlertsDf1 = trafficAlertsDf[[col for col in trafficAlertsDf if col not in ['coordType','coordinates']]+['coordType','coordinates']] 

In [16]:
trafficAlertsDf1

,alertID,pubMillis,type,subtype,street,city,confidence,reliability,datumCas,datum,weekdays,hours,minutes,quarters,coordType,coordinates
0,f5987a0c-c5e0-3064-ad5d-9d58b296b1d9,2020/07/12 11:35:50+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,U Dráhy,Ostopovice,0,7,2020-07-12 13:35:50+02:00,2020-07-12,Sunday,13,35,30,Point,"[16.544367, 49.16387]"
1,da8343e6-c5f7-3478-a969-47fb8460c2bb,2020/07/13 03:20:19+00,WEATHERHAZARD,HAZARD_ON_ROAD_CONSTRUCTION,D1,NaN,5,10,2020-07-13 05:20:19+02:00,2020-07-13,Monday,5,20,15,Point,"[16.417239, 49.209704]"
2,f5957426-0522-3b9d-b6ba-81c44c8aa182,2020/07/12 10:33:18+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,Průhon,Brno,1,8,2020-07-12 12:33:18+02:00,2020-07-12,Sunday,12,33,30,Point,"[16.54785, 49.20859]"
3,84b2780e-43bc-3883-81f9-a54a451efcbb,2020/07/12 15:09:24+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,NaN,Brno,0,6,2020-07-12 17:09:24+02:00,2020-07-12,Sunday,17,9,00,Point,"[16.608269, 49.18394]"
4,c2b5e9f5-8eef-3e08-9092-ba52f4726a62,2020/07/12 10:51:41+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,NaN,Brno,0,6,2020-07-12 12:51:41+02:00,2020-07-12,Sunday,12,51,45,Point,"[16.565195, 49.168989]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86600,55f0e244-a526-43c5-b3b0-53650b42bbde,2020/10/14 13:16:20+00,WEATHERHAZARD,HAZARD_ON_ROAD_CONSTRUCTION,Cejl,Brno,5,10,2020-10-14 15:16:20+02:00,2020-10-14,Wednesday,15,16,15,Point,"[16.618298, 49.197643]"
86601,7d5da69e-874f-4d4a-9bf5-d7aa99efc0e8,2020/10/14 12:30:54+00,WEATHERHAZARD,HAZARD_ON_ROAD_CONSTRUCTION,Bělohorská,Brno,5,10,2020-10-14 14:30:54+02:00,2020-10-14,Wednesday,14,30,15,Point,"[16.666439, 49.186837]"
86602,7cbf54fd-18cb-4377-b2dd-b7a4fde9df63,2020/10/14 17:00:45+00,WEATHERHAZARD,HAZARD_ON_SHOULDER_CAR_STOPPED,Křížová,Brno,0,5,2020-10-14 19:00:45+02:00,2020-10-14,Wednesday,19,0,00,Point,"[16.594498, 49.18925]"
86603,312fae72-4d8a-4ef7-be13-57138be7c4a9,2020/10/14 07:15:52+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,374,NaN,0,6,2020-10-14 09:15:52+02:00,2020-10-14,Wednesday,9,15,00,Point,"[16.664688, 49.268111]"


## Geocoding

In [17]:
alertStreetsNaNDf = trafficAlertsDf1[trafficAlertsDf1['street'].isna()] #tabulka chybějících ulic

In [18]:
type(alertStreetsNaNDf.iloc[0,-1])

str

In [19]:
len(alertStreetsNaNDf)

3804

In [20]:
NaNStreetList = alertStreetsNaNDf.values.tolist()

In [21]:
souradnice = [ast.literal_eval(x[-1]) for x in NaNStreetList] #vytvoření seznamu seznamu souřadnic
souradnice[1:5]

[[16.565195, 49.168989],
 [16.637137, 49.184795],
 [16.603667, 49.196316],
 [16.637025, 49.184858]]

In [22]:
geopy.geocoders.options.default_user_agent = 'Mozilla/5.0'
geolocator = Nominatim()

In [ ]:
AlertStreetList = [] #doplněný seznam pomocí násled.cyklu,

In [ ]:
for i in range(0,len(souradnice)):
    reverse = RateLimiter(geolocator.reverse, min_delay_seconds = 1)
    location = reverse([souradnice[i][1],souradnice[i][0]], language = 'cz')#pozor potřeba otočit pořadí souřadnic
    if 'road' in location.raw['address']:
        name = location.raw['address']['road']
    elif 'suburb' in location.raw['address']:
        suburb = location.raw['address']['suburb']
        municip = location.raw['address']['municipality']
        name = suburb+','+municip
    else:
        suburb = 'missing'
        municip = location.raw['address']['municipality']
        name = suburb+','+municip
    AlertStreetList.append(name)

In [ ]:
alertStreetsNaNDf['missing_street'] = AlertStreetList
alertStreetsNaNDf.head()

In [ ]:
alertStreetsNaNDf.to_csv('alertStreetNaNDf.csv', encoding = 'UTF-8', index = False)

In [23]:
alertStreetsNaNDf = pd.read_csv('alertStreetNaNDf.csv', encoding = 'UTF-8')#.drop(['Unnamed: 0'], axis = 1)
alertStreetsNaNDf.head()

,Unnamed: 0,alertID,pubMillis,type,subtype,roadType,street,city,country,objectid,globalid,datumCas,weekdays,hours,quarters,minutes,coordType,coordinates,missing_street
0,3,84b2780e-43bc-3883-81f9-a54a451efcbb,2020/07/12 15:09:24+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,NaN,NaN,Brno,EZ,4,{39CFFD83-039F-C700-646A-85DF84936E25},2020-07-12 15:09:24,Sunday,15,1,9,Point,"[16.608269, 49.18394]",Uhelná
1,4,c2b5e9f5-8eef-3e08-9092-ba52f4726a62,2020/07/12 10:51:41+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,NaN,NaN,Brno,EZ,5,{DD1FB7D7-1967-F229-3D7A-E22413453428},2020-07-12 10:51:41,Sunday,10,4,51,Point,"[16.565195, 49.168989]",Okrouhlá
2,28,a74136e0-cfed-3712-a9e3-10a5d267d86a,2020/06/06 21:38:08+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,NaN,NaN,Brno,EZ,29,{BE285DC6-81DB-FFC8-DF50-F350C8BACFDA},2020-06-06 21:38:08,Saturday,21,3,38,Point,"[16.637137, 49.184795]",Charbulova
3,31,8509aa8c-e0f5-3043-b14d-1cbaf92e44f0,2020/07/02 20:29:08+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,NaN,NaN,Brno,EZ,32,{B92419CC-704B-1DA1-1C74-0447837F7CEC},2020-07-02 20:29:08,Thursday,20,2,29,Point,"[16.603667, 49.196316]",Údolní
4,40,1030b46a-1157-3fd6-96fc-66e078bf59ff,2020/07/12 11:36:51+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,NaN,NaN,Brno,EZ,41,{2B448056-680F-CAE0-3DD2-432DDA2B364F},2020-07-12 11:36:51,Sunday,11,3,36,Point,"[16.637025, 49.184858]",Charbulova


In [24]:
missAlertCoord = []
for i in alertStreetsNaNDf.index:
    if 'okres' in alertStreetsNaNDf['missing_street'][i]:
        missAlertCoord.append(alertStreetsNaNDf['coordinates'][i])

In [25]:
len(missAlertCoord)

1083

druhý způsob přiřazení názvu podle souřadnic - nevyhledal další ulice...smazáno

spojení tabulky trafficAlertsDf1 s tabulkou doplněných názvů

In [26]:
alertStreetsNaNDf2 = alertStreetsNaNDf[['alertID', 'missing_street']]

In [27]:
trafficAlertsDfMerge = trafficAlertsDf1.merge(alertStreetsNaNDf2,how = 'left', on = 'alertID')

In [28]:
trafficAlertsDfMerge.head()

,alertID,pubMillis,type,subtype,street,city,confidence,reliability,datumCas,datum,weekdays,hours,minutes,quarters,coordType,coordinates,missing_street
0,f5987a0c-c5e0-3064-ad5d-9d58b296b1d9,2020/07/12 11:35:50+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,U Dráhy,Ostopovice,0,7,2020-07-12 13:35:50+02:00,2020-07-12,Sunday,13,35,30,Point,"[16.544367, 49.16387]",NaN
1,da8343e6-c5f7-3478-a969-47fb8460c2bb,2020/07/13 03:20:19+00,WEATHERHAZARD,HAZARD_ON_ROAD_CONSTRUCTION,D1,NaN,5,10,2020-07-13 05:20:19+02:00,2020-07-13,Monday,5,20,15,Point,"[16.417239, 49.209704]",NaN
2,f5957426-0522-3b9d-b6ba-81c44c8aa182,2020/07/12 10:33:18+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,Průhon,Brno,1,8,2020-07-12 12:33:18+02:00,2020-07-12,Sunday,12,33,30,Point,"[16.54785, 49.20859]",NaN
3,84b2780e-43bc-3883-81f9-a54a451efcbb,2020/07/12 15:09:24+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,NaN,Brno,0,6,2020-07-12 17:09:24+02:00,2020-07-12,Sunday,17,9,00,Point,"[16.608269, 49.18394]",Uhelná
4,c2b5e9f5-8eef-3e08-9092-ba52f4726a62,2020/07/12 10:51:41+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,NaN,Brno,0,6,2020-07-12 12:51:41+02:00,2020-07-12,Sunday,12,51,45,Point,"[16.565195, 49.168989]",Okrouhlá


doplnění názvů do sloupce street

In [29]:
trafficAlertsDfMerge['street'] = trafficAlertsDfMerge['street'].fillna("")

In [30]:
streetNew = []
for i in trafficAlertsDfMerge.index:
    if len(trafficAlertsDfMerge['street'][i]) == 0:
        streetNew.append(trafficAlertsDfMerge['missing_street'][i])
    else:
        streetNew.append(trafficAlertsDfMerge['street'][i])

In [31]:
trafficAlertsDfMerge['streetNew'] = streetNew
trafficAlertsDfMerge.head()

,alertID,pubMillis,type,subtype,street,city,confidence,reliability,datumCas,datum,weekdays,hours,minutes,quarters,coordType,coordinates,missing_street,streetNew
0,f5987a0c-c5e0-3064-ad5d-9d58b296b1d9,2020/07/12 11:35:50+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,U Dráhy,Ostopovice,0,7,2020-07-12 13:35:50+02:00,2020-07-12,Sunday,13,35,30,Point,"[16.544367, 49.16387]",NaN,U Dráhy
1,da8343e6-c5f7-3478-a969-47fb8460c2bb,2020/07/13 03:20:19+00,WEATHERHAZARD,HAZARD_ON_ROAD_CONSTRUCTION,D1,NaN,5,10,2020-07-13 05:20:19+02:00,2020-07-13,Monday,5,20,15,Point,"[16.417239, 49.209704]",NaN,D1
2,f5957426-0522-3b9d-b6ba-81c44c8aa182,2020/07/12 10:33:18+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,Průhon,Brno,1,8,2020-07-12 12:33:18+02:00,2020-07-12,Sunday,12,33,30,Point,"[16.54785, 49.20859]",NaN,Průhon
3,84b2780e-43bc-3883-81f9-a54a451efcbb,2020/07/12 15:09:24+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,,Brno,0,6,2020-07-12 17:09:24+02:00,2020-07-12,Sunday,17,9,00,Point,"[16.608269, 49.18394]",Uhelná,Uhelná
4,c2b5e9f5-8eef-3e08-9092-ba52f4726a62,2020/07/12 10:51:41+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,,Brno,0,6,2020-07-12 12:51:41+02:00,2020-07-12,Sunday,12,51,45,Point,"[16.565195, 49.168989]",Okrouhlá,Okrouhlá


In [32]:
trafficAlertsDfFinal = trafficAlertsDfMerge.iloc[:,[0,1,2,3,17,5,6,7,8,13,9,10,12,11,14,15]]

rozdělení souřadnic do samostatných sloupečků

In [ ]:
#trafficAlertsDfFinal = pd.read_csv('trafficAlertsFinal.csv', encoding = 'utf-8')

In [33]:
coordinatesList = [ast.literal_eval(coord) for coord in trafficAlertsDfFinal['coordinates']] #vytvoření seznamu seznamu souřadnic
coordinatesList[:10]

[[16.544367, 49.16387],
 [16.417239, 49.209704],
 [16.54785, 49.20859],
 [16.608269, 49.18394],
 [16.565195, 49.168989],
 [16.546977, 49.162737],
 [16.489767, 49.207562],
 [16.546835, 49.20878],
 [16.504498, 49.15467],
 [16.619149, 49.182862]]

In [34]:
coordinatesX = [x[0] for x in coordinatesList]
coordinatesY = [y[1] for y in coordinatesList]

In [ ]:
trafficAlertsDfFinal['coordinatesX'] = coordinatesX
trafficAlertsDfFinal['coordinatesY'] = coordinatesY

In [36]:
trafficAlertsDfFinal

,alertID,pubMillis,type,subtype,streetNew,city,confidence,reliability,datumCas,quarters,datum,weekdays,minutes,hours,coordType,coordinates,coordinatesX,coordinatesY
0,f5987a0c-c5e0-3064-ad5d-9d58b296b1d9,2020/07/12 11:35:50+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,U Dráhy,Ostopovice,0,7,2020-07-12 13:35:50+02:00,30,2020-07-12,Sunday,35,13,Point,"[16.544367, 49.16387]",16.544367,49.163870
1,da8343e6-c5f7-3478-a969-47fb8460c2bb,2020/07/13 03:20:19+00,WEATHERHAZARD,HAZARD_ON_ROAD_CONSTRUCTION,D1,NaN,5,10,2020-07-13 05:20:19+02:00,15,2020-07-13,Monday,20,5,Point,"[16.417239, 49.209704]",16.417239,49.209704
2,f5957426-0522-3b9d-b6ba-81c44c8aa182,2020/07/12 10:33:18+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,Průhon,Brno,1,8,2020-07-12 12:33:18+02:00,30,2020-07-12,Sunday,33,12,Point,"[16.54785, 49.20859]",16.547850,49.208590
3,84b2780e-43bc-3883-81f9-a54a451efcbb,2020/07/12 15:09:24+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,Uhelná,Brno,0,6,2020-07-12 17:09:24+02:00,00,2020-07-12,Sunday,9,17,Point,"[16.608269, 49.18394]",16.608269,49.183940
4,c2b5e9f5-8eef-3e08-9092-ba52f4726a62,2020/07/12 10:51:41+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,Okrouhlá,Brno,0,6,2020-07-12 12:51:41+02:00,45,2020-07-12,Sunday,51,12,Point,"[16.565195, 49.168989]",16.565195,49.168989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86600,55f0e244-a526-43c5-b3b0-53650b42bbde,2020/10/14 13:16:20+00,WEATHERHAZARD,HAZARD_ON_ROAD_CONSTRUCTION,Cejl,Brno,5,10,2020-10-14 15:16:20+02:00,15,2020-10-14,Wednesday,16,15,Point,"[16.618298, 49.197643]",16.618298,49.197643
86601,7d5da69e-874f-4d4a-9bf5-d7aa99efc0e8,2020/10/14 12:30:54+00,WEATHERHAZARD,HAZARD_ON_ROAD_CONSTRUCTION,Bělohorská,Brno,5,10,2020-10-14 14:30:54+02:00,15,2020-10-14,Wednesday,30,14,Point,"[16.666439, 49.186837]",16.666439,49.186837
86602,7cbf54fd-18cb-4377-b2dd-b7a4fde9df63,2020/10/14 17:00:45+00,WEATHERHAZARD,HAZARD_ON_SHOULDER_CAR_STOPPED,Křížová,Brno,0,5,2020-10-14 19:00:45+02:00,00,2020-10-14,Wednesday,0,19,Point,"[16.594498, 49.18925]",16.594498,49.189250
86603,312fae72-4d8a-4ef7-be13-57138be7c4a9,2020/10/14 07:15:52+00,ROAD_CLOSED,ROAD_CLOSED_EVENT,374,NaN,0,6,2020-10-14 09:15:52+02:00,00,2020-10-14,Wednesday,15,9,Point,"[16.664688, 49.268111]",16.664688,49.268111


In [37]:
trafficAlertsDfFinal.to_csv('trafficAlertsDfFinal.csv', encoding = 'UTF-8', index = False)

## Duplicity

In [38]:
trafficAlertsDfNoDupl=trafficAlertsDfFinal.copy()

In [39]:
trafficAlertsDfNoDupl=trafficAlertsDfNoDupl.sort_values(by = ['streetNew','subtype','datumCas'])

In [40]:
AlertsDuplList = trafficAlertsDfNoDupl.duplicated(subset = ['subtype', 'streetNew','datum']) #seznam True/False
AlertsDfDupl= trafficAlertsDfNoDupl[trafficAlertsDfNoDupl.duplicated(subset = ['subtype', 'streetNew','datum'])]
len(AlertsDfDupl) #tabulka splňující podmínku pro duplikáty

52908

formát souřadnic - tuple

In [41]:
trafficAlertsDfNoDupl['point'] = list(zip(trafficAlertsDfNoDupl['coordinatesX'], trafficAlertsDfNoDupl['coordinatesY']))

In [42]:
trafficAlertsDfNoDupl=trafficAlertsDfNoDupl.reset_index()


In [43]:
trafficAlertsDfNoDupl = trafficAlertsDfNoDupl.drop(labels = ['coordinates','coordType','index'], axis = 1)

In [ ]:
#distance.distance(trafficAlertsDfNoDupl['point'][12019],trafficAlertsDfNoDupl['point'][26000]).m

In [44]:
trafficAlertsDfNoDupl2=trafficAlertsDfNoDupl.copy()

odmazání duplicit

In [45]:
for i in range(1,len(trafficAlertsDfNoDupl2)):
    if (AlertsDuplList[i]) & (trafficAlertsDfNoDupl['datumCas'][i]-trafficAlertsDfNoDupl['datumCas'][i-1]< timedelta(minutes = 15))& (distance.distance(trafficAlertsDfNoDupl['point'][i],trafficAlertsDfNoDupl['point'][i-1]).km < 1):
        trafficAlertsDfNoDupl2 = trafficAlertsDfNoDupl2.drop([i])
        i = i-1

In [46]:
len(trafficAlertsDfNoDupl2)

78935

finální úpravy názvů street, city

In [47]:
trafficAlertsDfNoDupl2['city'] = trafficAlertsDfNoDupl2['city'].fillna('Brno')

In [48]:
trafficAlertsDfNoDupl2['streetNew'] = trafficAlertsDfNoDupl2['streetNew'].str.strip()

In [49]:
trafficAlertsDfNoDupl2['streetNew'] = trafficAlertsDfNoDupl2['streetNew'].replace("Žabovřesky,okres Brno-město","Žabovřeská") #na základě kontroly v mapě (606 nahrazení)

In [ ]:
trafficAlertsDfNoDupl2.head()

In [50]:
trafficAlertsDfNoDupl2.to_csv('trafficAlertsDfNoDupl1km.csv', encoding = 'UTF-8', index = False)

zgrupování po dnech a hodinách

In [51]:
AlertsGroupDf = pd.read_csv('trafficAlertsDfNoDupl1km.csv', encoding = 'utf-8')

In [53]:
AlertsGroupDfh = AlertsGroupDf.groupby(['datum','hours'])['alertID'].count()

In [54]:
AlertsGroupDfh

datum       hours
2020-04-21  5        14
2020-04-25  23        1
2020-04-28  14       42
2020-04-29  5         1
            19        1
                     ..
2020-10-14  15        4
            16        4
            17        7
            18       10
            19        2
Name: alertID, Length: 1966, dtype: int64

In [55]:
AlertsGroupDfh.to_csv('AlertsGroupDfh1.csv', encoding = 'UTF-8')